# Determinación de la Curva ROC y el Cut off

* Kernel Python 3.09.12

In [13]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-

# ! pip install --upgrade pip
# !pip install sinfo
# !pip install --upgrade numpy 
# !pip install plotly --upgrade
# !pip install chart-studio --upgrade
#!pip install ibm-cos-sdk #Documentación: https://ibm.github.io/ibm-cos-sdk-python/
# !pip install boto3

In [15]:
# Instalamos las librerias necesarias
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly as plotly
import pandas as pd
from botocore.client import Config
import numpy as np
# import numpy.dual as dual --> Deprecado
import scipy as sp
from sklearn import metrics
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3 #Documentación: https://ibm.github.io/ibm-cos-sdk-python/
def __iter__(self): return 0

import warnings
warnings.filterwarnings("ignore")

#Un-Comment these options if you want to exapand the number of rows and columns of you see visually in the notebook.
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


# Articulo

* Link: https://medium.com/swlh/determining-a-cut-off-or-threshold-when-working-with-a-binary-dependent-target-variable-7c2342cf2a7c

In [19]:
# Cargamos el dataset
pd_data = pd.read_csv("Franciscoletelier/Python-Projects/df_for_export-2.csv", sep=",", header=0)

FileNotFoundError: [Errno 2] No such file or directory: 'Franciscoletelier/Python-Projects/df_for_export-2.csv'

In [ ]:
# Find the minimum probability for each decile
tips_summedv = pd.DataFrame(dfx.groupby(['DECILE'])['P_FAIL'].min())
# Find the maximum probability of each decile
tips_summedw = pd.DataFrame(dfx.groupby(['DECILE'])['P_FAIL'].max())
# Find the Actual Failure rate for each decile.
tips_summedx = pd.DataFrame(dfx.groupby(['DECILE'])['FAILURE_TARGET'].mean())
#Sum the number of Failures in each decile.
tips_summedy = pd.DataFrame(dfx.groupby(['DECILE'])['FAILURE_TARGET'].sum())
# count the records in each decile
tips_summedz = pd.DataFrame(dfx.groupby(['DECILE'])['FAILURE_TARGET'].count())


In [ ]:
#Aggregate the summaries into one dataframe
tips = pd.concat([tips_summedv,tips_summedw, tips_summedx, tips_summedy,tips_summedz], axis=1)
tips.columns = ['MIN_SCORE','MAX_SCORE','FAILURE_RATE','FAILURES', 'OBS']


In [ ]:
tips=tips.sort_values(by=['DECILE'], ascending=[False])
gains=tips
#Find the number of cumulative failures by decile.
gains['CUML_FAILURES']=gains['FAILURES'].cumsum()
#Find the percentage of failures in each decile
gains['PCT_OF_FAILURES']=(gains.FAILURES)/(dfx['FAILURE_TARGET'].sum())*100
#Find the cumulative percentage of failures in each decile.
gains['CUML_PCT_OF_FAILURES']=gains.PCT_OF_FAILURES.cumsum()
#Format the final output
gains=gains[['OBS','MIN_SCORE','MAX_SCORE','FAILURES','FAILURE_RATE','PCT_OF_FAILURES','CUML_FAILURES','CUML_PCT_OF_FAILURES']]
gains

## 4.1 Finding the best cut-off

In [20]:
dfx=pd_data
#Add a small negative random number to P_FAIL to break ties when grouping
dfx['wookie'] = (np.random.randint(0, 100, dfx.shape[0]))/100000000000000000
dfx['P_FAIL']=dfx['P_FAIL']+dfx['wookie']

# Instead of deciles, we create 10000 groups, based on the probability to fail.

NameError: name 'pd_data' is not defined

In [ ]:
dfx['GROUPS'] = pd.qcut(dfx['P_FAIL'], 10000, labels=False)
# find the minimum P_FAIL for each group.  This is a potential cut-off point.
tips_summedb = pd.DataFrame(dfx.groupby(['GROUPS'])['P_FAIL'].min())
#Find the number of Failures in each group
tips_summedz = pd.DataFrame(dfx.groupby(['GROUPS'])['FAILURE_TARGET'].sum())
#find the number of observations in each group
tips_summeda = pd.DataFrame(dfx.groupby(['GROUPS'])['FAILURE_TARGET'].count())
#append the summaries into one dataframe
tips = pd.concat([tips_summedb,tips_summedz, tips_summeda], axis=1)
tips.columns = ['CUT-OFF','FAILURES', 'OBS']
#find the number of non-failures
tips['NON_FAILURES']=tips.OBS-tips.FAILURES
#reset the index to make GROUPS a column
tips.reset_index(level=0, inplace=True)
#sort the dataframe by groups in descending order
tips=tips.sort_values(by=['GROUPS'], ascending=[False])
# Cumulative sum the failures, non-failures and observations
tips['INV_CUM_FAILURES'] = tips.FAILURES.cumsum()
tips['INV_CUM_NON_FAILURES'] = tips.NON_FAILURES.cumsum()
tips['TOTAL_OBS']=tips.OBS.sum()
#Sort the data by Groups ascending
tips=tips.sort_values(by=['GROUPS'], ascending=[True])
#calculate the total number of failures and non-failures
tips['CUM_FAILURES'] = tips.FAILURES.cumsum()
tips['CUM_NON_FAILURES'] = tips.NON_FAILURES.cumsum()
#find the total number of failures for the whole dataset.
tips['TOTAL_FAILURES']=tips.FAILURES.sum()
tips['TOTAL_NON_FAILURES']=tips.NON_FAILURES.sum()
#define the true positives for each cut-off
tips['TRUE_POSITIVES']=tips.INV_CUM_FAILURES
#define the false positives for each cut-off
tips['FALSE_POSITIVES']=tips.INV_CUM_NON_FAILURES
#define the true negatives for each cut-off
tips['TRUE_NEGATIVES']=tips.CUM_NON_FAILURES-tips.NON_FAILURES
#define the false negatives for each cut-off
tips['FALSE_NEGATIVES']=tips.CUM_FAILURES-tips.FAILURES
#double check the logic and arithmetic.
tips['OBS2']=tips.TRUE_POSITIVES+tips.FALSE_POSITIVES+tips.TRUE_NEGATIVES+tips.FALSE_NEGATIVES
# define the sensitvity for each cut-off
tips['SENSITIVITY']=tips['TRUE_POSITIVES']/(tips['TRUE_POSITIVES']+tips['FALSE_NEGATIVES'])
#define the specificity for each cut-off
tips['SPECIFICITY']=tips['TRUE_NEGATIVES']/(tips['FALSE_POSITIVES']+tips['TRUE_NEGATIVES'])
#define the false positive rate for each cut-off
tips['FALSE_POSITIVE_RATE']=1-tips['SPECIFICITY']
#define the false negative rate for each cut-off
tips['FALSE_NEGATIVE_RATE']=1-tips['SENSITIVITY']
tipsx=tips
gains=tipsx[['GROUPS','CUT-OFF','TRUE_POSITIVES','FALSE_POSITIVES','TRUE_NEGATIVES','FALSE_NEGATIVES','SENSITIVITY',
            'SPECIFICITY','FALSE_POSITIVE_RATE','FALSE_NEGATIVE_RATE']]
gains

## 4.2 Finding the cut-off with the smallest misclassification rate

In [ ]:
tips=tipsx
#sum the false positives and false negatives.
tips['FALSE_CLASSIFICATIONS'] = tips.FALSE_POSITIVES+tips.FALSE_NEGATIVES
#estimate the false classification rate
tips['FALSE_CLASSIFICATION_RATE']=tips.FALSE_CLASSIFICATIONS/(tips.TOTAL_OBS)
gains=tips[['GROUPS','CUT-OFF','TRUE_POSITIVES','FALSE_POSITIVES','TRUE_NEGATIVES','FALSE_NEGATIVES','SENSITIVITY',
            'SPECIFICITY','FALSE_POSITIVE_RATE','FALSE_NEGATIVE_RATE','FALSE_CLASSIFICATIONS','FALSE_CLASSIFICATION_RATE']]
gains

In [ ]:
x1 = tips['CUT-OFF']
y1 = tips['FALSE_POSITIVE_RATE']
y2 = tips['FALSE_NEGATIVE_RATE']
y3 = tips['FALSE_CLASSIFICATION_RATE']
trace = go.Scatter(
    x = x1,
    y = y1,
    name='False Positive Rate')
trace2 = go.Scatter(
    x = x1,
    y = y2,
    name='False Negative Rate'
)
trace3 = go.Scatter(
    x = x1,
    y = y3,
    name='False Classification Rate'
)
layout = go.Layout(
    title='Mis-Classification Rates BY CUT OFF SCORE',
    xaxis=dict(
        title='CUT OFF SCORE',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='False Positive and False Negative Rates',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=True,
)
    
data=[trace,trace2,trace3]  
fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='styling-names')
plotly.offline.iplot(fig, filename='shapes-lines')

In [ ]:
gains=gains.sort_values(by=['FALSE_CLASSIFICATION_RATE'], ascending=[True])
gains=gains.head(1)
gains

In [ ]:
dfx=pd_data
dfx[‘Y_FAIL’] = np.where(((dfx.P_FAIL <= .909427)), 0, 1)
print(pd.crosstab(dfx.Y_FAIL, dfx.FAILURE_TARGET, dropna=False))
pd.crosstab(dfx.Y_FAIL, dfx.FAILURE_TARGET).apply(lambda r: r/r.sum(), axis=1)

In [ ]:
dfx=pd_data
dfx['Y_FAIL'] = np.where(((dfx.P_FAIL <= .5)), 0, 1)
print(pd.crosstab(dfx.Y_FAIL, dfx.FAILURE_TARGET, dropna=False))
pd.crosstab(dfx.Y_FAIL, dfx.FAILURE_TARGET).apply(lambda r: r/r.sum(), axis=1)


In [ ]:
x2 = tips['CUT-OFF']
x1 = tips['FALSE_POSITIVE_RATE']
y1 = tips['SENSITIVITY']
trace = go.Scatter(
    x = x1,
    y = y1,
    name='ROC')
trace2 = go.Scatter(
    x = x2,
    y = y1,
    name='Cut-Off v TPR'
)
layout = go.Layout(
    title='ROC with Threshold Levels',
    xaxis=dict(
        title='False Positive Rate and Threshold Level',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='True Positive Rate',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=True,
)
    
data=[trace,trace2]  
fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='styling-names')
plotly.offline.iplot(fig, filename='shapes-lines')

## 4.3 Finding the cut-off when the costs of false positives and false negatives are not equal

In [ ]:
tips=tipsx
#define the cost of a false positive and false negative
cost_of_a_false_positive=1
cost_of_a_false_negative=2
#convert the costs into weights that sum to one
fp_weight=(cost_of_a_false_positive)/(cost_of_a_false_positive+cost_of_a_false_negative)
fn_weight=(cost_of_a_false_negative)/(cost_of_a_false_positive+cost_of_a_false_negative)
#Create a weighted false classification rate based on the costs of a false positive and a false negative.
tips['FALSE_CLASSIFICATIONS_W'] = np.where(((2*(((fp_weight)*tips.FALSE_POSITIVES+(fn_weight)*tips.FALSE_NEGATIVES)) >= tips.TOTAL_OBS)), tips.TOTAL_OBS, 2*(((fp_weight)*tips.FALSE_POSITIVES+(fn_weight)*tips.FALSE_NEGATIVES)))
tips['FALSE_CLASSIFICATION_RATE_W']=tips.FALSE_CLASSIFICATIONS_W/(tips.TOTAL_OBS)
gains=tips[['GROUPS','CUT-OFF','TRUE_POSITIVES','FALSE_POSITIVES','TRUE_NEGATIVES','FALSE_NEGATIVES','SENSITIVITY',
            'SPECIFICITY','FALSE_POSITIVE_RATE','FALSE_NEGATIVE_RATE','FALSE_CLASSIFICATIONS_W','FALSE_CLASSIFICATION_RATE_W']]
gains

In [ ]:
x1 = tips['CUT-OFF']
y1 = tips['FALSE_POSITIVE_RATE']
y2 = tips['FALSE_NEGATIVE_RATE']
y3 = tips['FALSE_CLASSIFICATION_RATE_W']
trace = go.Scatter(
    x = x1,
    y = y1,
    name='False Positive Rate')
trace2 = go.Scatter(
    x = x1,
    y = y2,
    name='False Negative Rate'
)
trace3 = go.Scatter(
    x = x1,
    y = y3,
    name='Weighted False Classification Rate'
)
layout = go.Layout(
    title='Weighted Mis-Classification Rates BY CUT OFF SCORE',
    xaxis=dict(
        title='CUT OFF SCORE',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Weighted False Positive and False Negative Rates',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=True,
)
    
data=[trace,trace2,trace3]  
fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='styling-names')
plotly.offline.iplot(fig, filename='shapes-lines')

In [ ]:
gains=gains.sort_values(by=['FALSE_CLASSIFICATION_RATE_W'], ascending=[True])
gains=gains.head(1)
gains

In [ ]:
dfx=pd_data
dfx[‘Y_FAIL’] = np.where(((dfx.P_FAIL <= .8802)), 0, 1)
print(pd.crosstab(dfx.Y_FAIL, dfx.FAILURE_TARGET, dropna=False))
pd.crosstab(dfx.Y_FAIL, dfx.FAILURE_TARGET).apply(lambda r: r/r.sum(), axis=1)

## 4.4 Finding a cut-off when the economic costs are known.

In [ ]:
#Define the False Positive and False Negative Weights.
cost_of_a_false_positive=2500
cost_of_a_false_negative=27500
fp_weight=(cost_of_a_false_positive)/(cost_of_a_false_positive+cost_of_a_false_negative)
fn_weight=(cost_of_a_false_negative)/(cost_of_a_false_positive+cost_of_a_false_negative)
#Define the False Classification Rate
tips['FALSE_CLASSIFICATIONS_W'] = np.where(((2*(((fp_weight)*tips.FALSE_POSITIVES+(fn_weight)*tips.FALSE_NEGATIVES)) >= tips.TOTAL_OBS)), tips.TOTAL_OBS, 2*(((fp_weight)*tips.FALSE_POSITIVES+(fn_weight)*tips.FALSE_NEGATIVES)))
tips['FALSE_CLASSIFICATION_RATE_W']=tips.FALSE_CLASSIFICATIONS_W/(tips.TOTAL_OBS)
tips['TOTAL_COST']=tips.FALSE_POSITIVES*cost_of_a_false_positive+tips.FALSE_NEGATIVES*cost_of_a_false_negative
gains=tips[['GROUPS','CUT-OFF','TRUE_POSITIVES','FALSE_POSITIVES','TRUE_NEGATIVES','FALSE_NEGATIVES','SENSITIVITY',
            'SPECIFICITY','FALSE_POSITIVE_RATE','FALSE_NEGATIVE_RATE','FALSE_CLASSIFICATIONS_W','FALSE_CLASSIFICATION_RATE_W','TOTAL_COST']]
gains

In [ ]:
x1 = tips['CUT-OFF']
y1 = tips['FALSE_POSITIVE_RATE']
y2 = tips['FALSE_NEGATIVE_RATE']
y3 = tips['FALSE_CLASSIFICATION_RATE_W']
trace = go.Scatter(
    x = x1,
    y = y1,
    name='False Positive Rate')
trace2 = go.Scatter(
    x = x1,
    y = y2,
    name='False Negative Rate'
)
trace3 = go.Scatter(
    x = x1,
    y = y3,
    name='Weighted False Classification Rate'
)
layout = go.Layout(
    title='Weighted Mis-Classification Rates BY CUT OFF SCORE',
    xaxis=dict(
        title='CUT OFF SCORE',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='False Positive and False Negative Rates',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=True,
)
    
data=[trace,trace2,trace3]  
fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='styling-names')
plotly.offline.iplot(fig, filename='shapes-lines')

#### Let’s examine the relationship between Total Costs and the cut-off or threshold level

In [ ]:
x1 = tips['CUT-OFF']
y1 = tips['TOTAL_COST']
trace = go.Scatter(
    x = x1,
    y = y1,
    name='Total Cost')
layout = go.Layout(
    title='Cut-Off Levels and Total Cost',
    xaxis=dict(
        title='CUT OFF SCORE',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Total Costs',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=True,
)
    
data=[trace]  
fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='styling-names')
plotly.offline.iplot(fig, filename='shapes-lines')

In [ ]:
gains=gains.sort_values(by=['TOTAL_COST'], ascending=[True])
gains=gains.head(1)
gains

In [ ]:
dfx['Y_FAIL'] = np.where(((dfx.P_FAIL <= .665541)), 0, 1)
print(pd.crosstab(dfx.Y_FAIL, dfx.FAILURE_TARGET, dropna=False))
pd.crosstab(dfx.Y_FAIL, dfx.FAILURE_TARGET).apply(lambda r: r/r.sum(), axis=1)

## 5.0 Summary and conclusions
* As you can see from these three scenarios, the best cut-off will depend greatly on the economics of your problem. Because of this, it is important to understand economic costs of both a false positive and a false negative. Just like most things in data science. The “best” answer depends on the context. If a false negative means a bolt will fly off and poke someone’s eye out, you have to make sure your model predictions reflect this hazard. Data science, like all things in this world, is subject to the context in which it is applied